# Convert Coco JSON Annotations to YOLO TXT Files
Use this notebook to try out importing, analyzing, and exporting datasets of image annotations. 

In [7]:
import logging
logging.getLogger().setLevel(logging.CRITICAL)
import sys
!{sys.executable} -m pip install pylabel
from pylabel import importer

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


## Import coco annotations 
First we will import annotations from the coco dataset, which are in coco json format. 

In [8]:
path_to_annotations = "coco_labels_from_labeling_edited.json"
path_to_images = "../images"
dataset = importer.ImportCoco(path_to_annotations, path_to_images=path_to_images, name="labeled_data_coco_format")


## Visualize Annotations 
You can render the bounding boxes for your image to inspect them and confirm that they imported correctly.  

### Fikser sentrering av bokser

In [9]:
data = [[] for _ in range(4)]
for index, row in dataset.df.iterrows():
    xmin = min(row["ann_bbox_xmin"], row["ann_bbox_xmax"])
    ymin = min(row["ann_bbox_ymin"], row["ann_bbox_ymax"])
    data[0].append(xmin + (abs(row["ann_bbox_xmin"] - row["ann_bbox_xmax"])/2))
    data[1].append(ymin + (abs(row["ann_bbox_ymin"] - row["ann_bbox_ymax"])/2))
    data[2].append((abs(row["ann_bbox_xmin"] - row["ann_bbox_xmax"])))
    data[3].append((abs(row["ann_bbox_ymin"] - row["ann_bbox_ymax"])))

dataset.df["center_x_scaled"] = data[0]
dataset.df["center_y_scaled"] = data[1]
dataset.df["width_scaled"] = data[2]
dataset.df["height_scaled"] = data[3]

In [10]:
img_id = 0
bbox_df = dataset.df.loc[dataset.df['img_filename'] == f'{img_id}.png']
print(bbox_df)

     img_folder img_filename img_path  img_id  img_width  img_height  \
id                                                                     
3375  ../images        0.png              465        343         343   
3376  ../images        0.png              465        343         343   
3377  ../images        0.png              465        343         343   
3378  ../images        0.png              465        343         343   
3379  ../images        0.png              465        343         343   
3380  ../images        0.png              465        343         343   
3381  ../images        0.png              465        343         343   
3382  ../images        0.png              465        343         343   
3383  ../images        0.png              465        343         343   
3384  ../images        0.png              465        343         343   
3385  ../images        0.png              465        343         343   
3386  ../images        0.png              465        343        

In [128]:
import cv2

IMAGE_SIZE = 320
# Set new image size in pandas
dataset.df["img_width"] = IMAGE_SIZE
dataset.df["img_height"] = IMAGE_SIZE

# Open and resize image
img = cv2.imread(f"../images/cloud_cover_lt10/{img_id}.png")
img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE), interpolation=cv2.INTER_LINEAR)

# Print bounding boxes
bounding_boxes = []
for index, row in bbox_df.iterrows():
    x = int(row["center_x_scaled"]*IMAGE_SIZE-row["width_scaled"]*IMAGE_SIZE/2)
    y = int(row["center_y_scaled"]*IMAGE_SIZE-row["height_scaled"]*IMAGE_SIZE/2)
    w = int(row["width_scaled"]*IMAGE_SIZE)
    h = int(row["height_scaled"]*IMAGE_SIZE)
    bounding_boxes.append((x, y, x+w, (y+h)))

## Segmentation

In [ ]:
import numpy as np
import cv2

In [189]:
# Copy image
seg = img.copy()
cv2.cvtColor(seg, cv2.COLOR_BGR2RGB)

# Cut bounding box
xmin, ymin, xmax, ymax = bounding_boxes[10]
cv2.rectangle(seg, (xmin, ymin), (xmax, ymax), (255, 255, 255), 2)
seg = seg[ymin:ymax, xmin:xmax]
cv2.imwrite("./coral_bounding_box.png", seg)

mask = np.zeros(seg.shape[:2],np.uint8)
bgdModel = np.zeros((1,65),np.float64)
fgdModel = np.zeros((1,65),np.float64)
cv2.grabCut(seg, mask, (10, seg.shape[0]-10, 10, seg.shape[1]-1), bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)
mask2 = np.where((mask==2)|(mask==0), 255, 0).astype('uint8')
seg = seg*mask2[:,:,np.newaxis]
cv2.imwrite("./coral_segmented.png", seg)



True

In [ ]:
seg = img.copy()
seg = cv2.cvtColor(seg, cv2.COLOR_BGR2GRAY)
for xmin, ymin, xmax, ymax in bounding_boxes:
    mask = np.zeros(seg.shape[:2],np.uint8)
    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)
    # cv2.rectangle(seg, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)
    cv2.grabCut(seg, mask, (xmin,xmax,ymin,ymax), bgdModel, fgdModel, 1, cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask==2)|(mask==0), 255, 0).astype('uint8')
    seg = seg[0]*mask2[:,:,np.newaxis]

In [132]:
#Load the Image
seg = img.copy()
height, width = seg.shape[:2]

#Create a mask holder
mask = np.zeros(seg.shape[:2],np.uint8)

#Grab Cut the object
bgdModel = np.zeros((1,65),np.float64)
fgdModel = np.zeros((1,65),np.float64)

# Hard Coding the Rect… The object must lie within this rect.
rect = bounding_boxes[8]
cv2.grabCut(seg,mask,rect,bgdModel,fgdModel,20,cv2.GC_INIT_WITH_RECT)
mask = np.where((mask==2)|(mask==0),0,1).astype('uint8')
img1 = seg*mask[:,:,np.newaxis]

#Get the background
background = seg - img1

#Change all pixels in the background that are not black to white
background[np.where((background > [0,0,0]).all(axis = 2))] = [255,255,255]

#Add the background and the image
final = background + img1

cv2.imwrite("./segmented_0_320px.png", final)

True